# Извлечение признаков из изображений

In [ ]:
!pip install --upgrade -q gdown
!pip install html2text -q
!pip install timm -q

!mkdir /content/data/
!gdown 14dYvGXRVD1Prtr7SXHrwO6ZpfGk0F5aT
!unzip /content/internship_2023.zip -d /content/data/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 7.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from ast import literal_eval
from html2text import HTML2Text

from tqdm import tqdm
from glob import glob
import os

import timm
from PIL import Image
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [ ]:
class ProductImagesDataset(Dataset):
    """Датасет изображений продуктов
        Вход:
        root_dir - директория, в которой располагаются фотографии
        transform - желаемое преобразование изображений"""
    def __init__(self, root_dir : str, transform=None):
        self.img_list = glob(root_dir + '/*.jpg')
        self.root_dir = root_dir
        self.transform = transform
    

    def __len__(self):
        return len(self.img_list)

    # возвращает tuple из product_id и изображения
    def __getitem__(self, idx):
        im_path = self.img_list[idx]
        product_id = int(os.path.basename(im_path)[:-4])
        im = Image.open(im_path)

        if self.transform is not None:
            return (product_id, self.transform(im))
        return (product_id, im)

In [ ]:
def create_image_features(img_data, img_feature_extr, device, path='/content'):
    """
    Создает таблицу признаков изображений, в последнем столбце - product_id
    Вход:
     img_data - датасет фотографий
     img_feature_extr - feature_extractor изображений
     dataframe - табличные данные 
     path - директория, в которую сохранится таблица признаков
    Выход:
     Сохраняет dataframe мета признаков изображений"""

    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

    features_df = []
    loader = DataLoader(img_data, batch_size=5, shuffle=False, num_workers=2)

    print("Обработка изображений")
    for product_id, im in tqdm(loader):
        # получение фичей
        features = img_model(im.to(device))

        # запись в массив
        for i, out in enumerate(features):
            features_df.append(out.tolist() + [product_id[i].tolist()])
    
    # сохранение
    np.save(path + '/image_features', features_df)

In [ ]:
# модель, которая принимает на вход изображение, а возвращает вектор признаков
img_model = timm.create_model('maxxvit_rmlp_nano_rw_256', num_classes=0, pretrained=True)
img_model.to(device)
img_model.eval()

# трансформация изображение в тензор и нормализация
transform = transforms.Compose([transforms.PILToTensor(),
                                transforms.ConvertImageDtype(torch.float),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# инициализация данных
train = pd.read_parquet("/content/data/train.parquet")
image_dataset = ProductImagesDataset('/content/data/images/train', transform=transform)

# создание фичей
create_image_features(image_dataset, img_model)

<ipython-input-7-1ed35acd0af0>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  features_df[f'mf_{i}'] = ''


Обработка изображений


100%|██████████| 18224/18224 [2:48:13<00:00,  1.81it/s]


Изначально я сохранял данные в pd.DataFrame, но из-за сильного падения кол-ва итераций по мере заполнения датафрейма пришлось переключиться на формат npy. Именно поэтому фичи для train и test подгружаются в разных форматах.

## Проверка валидности признаков

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
train = pd.read_parquet("/content/data/train.parquet")
img_features = pd.read_csv('/content/data/features.csv')

# объединение данных по product_id
data = pd.merge(train, feat, on='product_id')

# удаление ненужных столбцов
data.drop(['product_id','sale','shop_id','shop_title','rating', 'text_fields', 'category_name'], axis=1, inplace=True)

In [ ]:
X = data[list(set(data.columns) - set(['category_id']))]
y = data['category_id']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# иногда некоторые классы не попадают в train выборку
# при финальном обучении будет использоваться стратификация
assert len(y_train.unique()) == 874, 'Не все классы попали в train выборку'

In [ ]:
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)

In [ ]:
print("F1 score:", f1_score(y_test, model.predict(X_test), average='weighted'))

F1 score: 0.5168456250681353


Получили неплохое качество, используя только изображения. Это говорит о том, что признаки отражают суть классов.

# Извлечение признаков из описания

In [ ]:
!pip install --upgrade -q gdown
!pip install html2text -q
!pip install transformers -q

!mkdir /content/data/
!gdown 14dYvGXRVD1Prtr7SXHrwO6ZpfGk0F5aT
!unzip /content/internship_2023.zip -d /content/data/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
from html2text import HTML2Text

import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

In [ ]:
def parse_text_fields(df, parser):
    """ Функция, которая преобразует text_fields в отдельные столбцы inplace
    df - табличные данные
    parser - парсер HTML кода"""
    
    # преобразование str в dict
    df['text_fields'] = df['text_fields'].apply(literal_eval)

    # преобразование text_fields в отдельные столбцы
    for key in df['text_fields'].iloc[0].keys():
        df[key] = df['text_fields'].apply(lambda x: x[key])

    # преобразовние descriprion из html в текст
    df['description'] = df['description'].apply(parser.handle)

    # очистка descriprion от лишних символов
    df['description'] = df['description'].apply(lambda x: x.replace('\n', ''))
    df['description'] = df['description'].apply(lambda x: x.replace('\\', ''))
    df['description'] = df['description'].apply(lambda x: x.replace('*', ''))


    df.drop('text_fields', axis=1, inplace=True)

In [ ]:
# парсинг text_fields
train = pd.read_parquet('/content/data/train.parquet')

parser = HTML2Text()
parser.escape_snob = True
parser.ignore_anchors = True
parser.skip_internal_links = True
parser.ignore_links = True
parser.ignore_images = True

parse_text_fields(train, parser)

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] 
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


def create_decription_features(model, tokenizer, df, path='/content'):
    """ Функция, которая создает векторное описание description и сохраняет
    полученные признаки в формате npy
    tokenizer - функция которая разбивает текст на блоки
    model - обрабатывает поступивший текст
    df - табличные данные
    path - путь для сохранения признаков"""

    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    print('Device:', device)
    
    sentences = df['description'].tolist()
    ids = df['product_id'].tolist()
    sentence_embeddings = []

    print("Извлечение признаков:")
    for sen in tqdm(sentences):
        encoded_input = tokenizer([sen], padding=True, truncation=True, max_length=120, return_tensors='pt').to(device)
        with torch.no_grad():
            model_output = model(**encoded_input)
        sentence_embeddings.append(mean_pooling(model_output, encoded_input['attention_mask']).tolist()[0])

    print("\n", 'Сохраниние признаков:')
    for i, emb in enumerate(sentence_embeddings):
        sentence_embeddings[i].append(ids[i])
    
    np.save(path + '/text_features', sentence_embeddings)
    print("Признаки сохранены в ", path)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")

create_decription_features(model, tokenizer, train)

Device: cuda:0
Извлечение признаков:


100%|██████████| 91120/91120 [42:57<00:00, 35.36it/s]



 Сохраниние признаков:
Признаки сохранены в  /content


## Проверка валидности признаков


Обучим KNN на новых признаках

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# загрузка данных
text = np.load('/content/text_features.npy')
train = pd.read_parquet('/content/data/train.parquet')

# инициализация столбцов текстовых признаков
columns=[]
for i in range(1024):
    columns.append(f'txt_mf_{i}')
columns.append('product_id')

# объеденение данных
text_features = pd.DataFrame(data=text,columns=columns)
data = pd.merge(text_features, train, on='product_id')

In [ ]:
X = data[list(set(data.columns.tolist()) - set(train.columns.tolist()))]
y = data['category_id']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# иногда некоторые классы не попадают в train выборку
assert len(y_train.unique()) == 874, 'Не все классы попали в train выборку'

In [ ]:
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)

In [ ]:
print("F1 score:", f1_score(y_test, model.predict(X_test), average='weighted'))

F1 score: 0.6532813226488928


Получили неплохое качестово, это значит, что признаковое описание отражает суть классов



# Извлечения признаков из названия продукта

Действуем по аналогии с извлечение признаков из description, отличая будут только в параметрах токенайзера, так как в title содержится меньше слов, чем в description.

In [ ]:
!pip install --upgrade -q gdown
!pip install html2text -q
!pip install transformers -q

!mkdir /content/data/
!gdown 14dYvGXRVD1Prtr7SXHrwO6ZpfGk0F5aT
!unzip /content/internship_2023.zip -d /content/data/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 1.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval

import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] 
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


def create_title_features(model, tokenizer, df, path='/content'):
    """ Функция, которая создает векторное описание title и сохраняет
    полученные признаки в формате npy
    tokenizer - функция которая разбивает текст на блоки
    model - обрабатывает поступивший текст
    df - табличные данные
    path - путь для сохранения признаков"""
    
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    print('Device:', device)

    model.to(device)

    sentences = df['title'].tolist()
    ids = df['product_id'].tolist()

    sentence_embeddings = []

    print("Извлечение признаков:")
    for sen in tqdm(sentences):
        encoded_input = tokenizer([sen], padding=True, truncation=True, max_length=15, return_tensors='pt').to(device)
        with torch.no_grad():
            model_output = model(**encoded_input)
        sentence_embeddings.append(mean_pooling(model_output, encoded_input['attention_mask']).tolist()[0])

    print("\n", 'Сохраниние признаков:')
    for i, emb in enumerate(sentence_embeddings):
        sentence_embeddings[i].append(ids[i])
    
    np.save(path + '/title_features', sentence_embeddings)
    print("Признаки сохранены в ", path)

In [ ]:
train = pd.read_parquet('/content/data/train.parquet')

def parse_text_fields(df):
    # преобразование str в dict
    df['text_fields'] = df['text_fields'].apply(literal_eval)

    # преобразование text_fields в отдельные столбцы
    for key in df['text_fields'].iloc[0].keys():
        df[key] = df['text_fields'].apply(lambda x: x[key])

parse_text_fields(train)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")

create_title_features(model, tokenizer, train)

Device: cuda:0
Извлечение признаков:


100%|██████████| 91120/91120 [31:32<00:00, 48.14it/s]



 Сохраниние признаков:
Признаки сохранены в  /content


## Проверка валидности признаков

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
title = np.load('/content/title_features.npy')
train = pd.read_parquet('/content/data/train.parquet')

columns=[]
for i in range(1024):
    columns.append(f'ttl_mf_{i}')
columns.append('product_id')

title_features = pd.DataFrame(data=title, columns=columns)
data = pd.merge(title_features, train, on='product_id')

In [ ]:
X = data[list(set(data.columns.tolist()) - set(train.columns.tolist()))]
y = data['category_id']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# иногда некоторые классы не попадают в train выборку
assert len(y_train.unique()) == 874, 'Не все классы попали в train выборку'

In [ ]:
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)
print("F1 score:", f1_score(y_test, model.predict(X_test), average='weighted'))

F1 score: 0.7445809076424476


# Объединение признаков и обучение


In [ ]:
!pip install --upgrade -q gdown 

!mkdir /content/data/
!mkdir /content/data/features

!gdown 14dYvGXRVD1Prtr7SXHrwO6ZpfGk0F5aT

%cd /content/data/features
!gdown 1OwHFAeESSHQ-wn4GUllww1FqHxA2tsSo 
!gdown 1cExxk3GT3NOY27osxcPJpkartjz6rZ71
!gdown 1SJR4gENw3wdJ9SmnQ6BeKdz7agFllAX6

!unzip /content/internship_2023.zip -d /content/data/

In [10]:
import pandas as pd
import numpy as np
from ast import literal_eval

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [11]:
im_features = pd.read_csv('/content/data/features/features.csv')
text_features = np.load('/content/data/features/text_features.npy')
title_features = np.load('/content/data/features/title_features.npy')
train = pd.read_parquet('/content/data/train.parquet')

In [12]:
def preprossesing(img_features, text_features, title_features, data):    
    """Вход:
        img_features - массив фичей изображений
        text_features - массив фичей описаний
        title_features - массив фичей названий
        data - табличные данные
      Выход:
        pd.DataFrame - сконкатинированные данные """
        
   # инициализация столбцов для text_features
    columns=[]
    for i in range(text_features.shape[1]-1):
        columns.append(f'txt_mf_{i}')
    columns.append('product_id')
    text_features = pd.DataFrame(data=text_features,columns=columns)


    # инициализация столбцов для title_features
    columns=[]
    for i in range(title_features.shape[1]-1):
        columns.append(f'ttl_mf_{i}')
    columns.append('product_id')
    title_features = pd.DataFrame(data=title_features,columns=columns)


    # преобразование str в dict
    data['text_fields'] = data['text_fields'].apply(literal_eval)

    # преобразование text_fields в отдельные столбцы
    for key in data['text_fields'].iloc[0].keys():
        data[key] = data['text_fields'].apply(lambda x: x[key])

    data.drop(['shop_id', 'shop_title', 'text_fields',
               'category_name', 'title', 'description', 'rating'], axis=1, inplace=True)
    
    # преобразуем некоторые признаки в категориальные
    data['attributes'] = data['attributes'].apply(lambda x: x==[])
    data['custom_characteristics'] = data['custom_characteristics'].apply(lambda x: x=={})
    data['defined_characteristics'] = data['defined_characteristics'].apply(lambda x: x=={})
    data['filters'] = data['filters'].apply(lambda x: x=={})

    data = pd.get_dummies(data, columns=['sale', 'attributes', 'custom_characteristics', 
                                         'defined_characteristics', 'filters'])
    
    data = pd.merge(data, text_features, on='product_id')
    data = pd.merge(data, img_features, on='product_id')
    data = pd.merge(data, title_features, on='product_id')
    data.drop('product_id', axis=1, inplace=True)

    return data

In [13]:
data = preprossesing(im_features, text_features, title_features, train)

# Сразу добавим дубликаты классов, которые имеют только один пример
data = pd.concat([data,
                  data[data['category_id']==11549],
                  data[data['category_id']==11875],
                  data[data['category_id']==12836],
                  data[data['category_id']==12901]])


X = data[list(set(data.columns.tolist()) - set(['category_id']))]
y = data['category_id']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.15, shuffle=True)

# очистим память
del im_features
del text_features
del title_features
del train
del data
del X
del y

In [14]:
model = KNeighborsClassifier()
model.fit(X_train, y_train)
print("KNN F1 score:", f1_score(y_test, model.predict(X_test), average='weighted'))

KNN F1 score: 0.7895025461703151


In [15]:
# эксперементы показали, что оптимально использовать один скрытый слой на 2500 нейронов
# большее количество итераций приводит к переобучению
model = MLPClassifier(hidden_layer_sizes=[2500], max_iter=10, verbose=True, early_stopping=True)
model.fit(X_train, y_train)
None

Iteration 1, loss = 1.40387280
Validation score: 0.795378
Iteration 2, loss = 0.53104911
Validation score: 0.816034
Iteration 3, loss = 0.36332067
Validation score: 0.813065
Iteration 4, loss = 0.26229001
Validation score: 0.820811
Iteration 5, loss = 0.19251708
Validation score: 0.822618
Iteration 6, loss = 0.15402187
Validation score: 0.823264
Iteration 7, loss = 0.12235820
Validation score: 0.826620
Iteration 8, loss = 0.10387153
Validation score: 0.824813
Iteration 9, loss = 0.10295827
Validation score: 0.815776
Iteration 10, loss = 0.10090559
Validation score: 0.824038


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


In [16]:
print("MLP F1 score:", f1_score(y_test, model.predict(X_test), average='weighted'))

MLP F1 score: 0.8275636571927997


# Submit

In [ ]:
!pip install --upgrade -q gdown
!pip install html2text -q
!pip install transformers -q
!pip install timm -q

!mkdir /content/data/
!gdown 14dYvGXRVD1Prtr7SXHrwO6ZpfGk0F5aT
!unzip /content/internship_2023.zip -d /content/data/

In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval
from html2text import HTML2Text

from tqdm import tqdm
from glob import glob
import os

import torch
import timm
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, AutoModel

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
PATH2SAVE = '/content/test_features'

# инициализация модели для обработки фотографий
img_model = timm.create_model('maxxvit_rmlp_nano_rw_256', num_classes=0, pretrained=True)
img_model.to(device)
img_model.eval()

transform = transforms.Compose([transforms.PILToTensor(),
                                transforms.ConvertImageDtype(torch.float),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# инициализация датафрейма
test = pd.read_parquet("/content/data/test.parquet")
# инициализация датасета фотографий, датасет был написан ранее
image_dataset = ProductImagesDataset('/content/data/images/test', transform=transform)

# создание признаков
create_image_features(image_dataset, img_model, device, PATH2SAVE)

Обработка изображений


100%|██████████| 3372/3372 [07:23<00:00,  7.61it/s]


In [ ]:
parser = HTML2Text()
parser.escape_snob = True
parser.ignore_anchors = True
parser.skip_internal_links = True
parser.ignore_links = True
parser.ignore_images = True

# преобразование текстовых данных функцией, которую инициализировали ранее
parse_text_fields(test, parser)

# инициализация токенайзера и модели для работы с текстом
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")

# создание признаков
# функции инициализировались ранее
create_decription_features(model, tokenizer, test, PATH2SAVE)
create_title_features(model, tokenizer, test, PATH2SAVE)

Device: cuda:0
Извлечение признаков:


100%|██████████| 16860/16860 [10:30<00:00, 26.75it/s]



 Сохраниние признаков:
Признаки сохранены в  /content/test_features
Device: cuda:0
Извлечение признаков:


100%|██████████| 16860/16860 [06:50<00:00, 41.06it/s]



 Сохраниние признаков:
Признаки сохранены в  /content/test_features


In [50]:
# подгрузка данных
image = np.load('/content/test_features/image_features.npy')
text = np.load('/content/test_features/text_features.npy')
title = np.load('/content/test_features/title_features.npy')
test = pd.read_parquet('/content/data/test.parquet')

In [51]:
# измененная функция препроцессинга, отличая:
# 1. Ожидаемый формат для img_features - np.array
# 2. Не удаляет столбец product_id 
def preprossesing_test(img_features, text_features, title_features, data):
    """Вход:
        img_features - массив фичей изображений
        text_features - массив фичей описаний
        title_features - массив фичей названий
        data - табличные данные
      Выход:
        pd.DataFrame - сконкатинированные данные """

    # инициализация столбцов для img_features
    columns=[]
    for i in range(img_features.shape[1]-1):
        columns.append(f'mf_{i}')
    columns.append('product_id')
    img_features = pd.DataFrame(data=img_features,columns=columns)    

   # инициализация столбцов для text_features
    columns=[]
    for i in range(text_features.shape[1]-1):
        columns.append(f'txt_mf_{i}')
    columns.append('product_id')
    text_features = pd.DataFrame(data=text_features,columns=columns)


    # инициализация столбцов для title_features
    columns=[]
    for i in range(title_features.shape[1]-1):
        columns.append(f'ttl_mf_{i}')
    columns.append('product_id')
    title_features = pd.DataFrame(data=title_features,columns=columns)

    # преобразование str в dict
    data['text_fields'] = data['text_fields'].apply(str).apply(literal_eval)

    # преобразование text_fields в отдельные столбцы
    for key in data['text_fields'].iloc[0].keys():
        data[key] = data['text_fields'].apply(lambda x: x[key])

    data.drop(['shop_id', 'shop_title', 'text_fields',
               'title', 'description', 'rating'], axis=1, inplace=True)

    
    # преобразуем некоторые признаки в категориальные
    data['attributes'] = data['attributes'].apply(lambda x: x==[])
    data['custom_characteristics'] = data['custom_characteristics'].apply(lambda x: x=={})
    data['defined_characteristics'] = data['defined_characteristics'].apply(lambda x: x=={})
    data['filters'] = data['filters'].apply(lambda x: x=={})

    data = pd.get_dummies(data, columns=['sale', 'attributes', 'custom_characteristics', 
                                         'defined_characteristics', 'filters'])

    data = pd.merge(data, text_features, on='product_id')
    data = pd.merge(data, img_features, on='product_id')
    data = pd.merge(data, title_features, on='product_id')

    return data

In [52]:
test_prep = preprossesing_test(image, text, title, test)
test_prep['predicted_category_id'] = model.predict(test_prep[X_train.columns])
test_prep[['product_id', 'predicted_category_id ']].to_parquet('/content/result.parquet')